# `MoveStrategy` and `MoveScheme`

After you've set up your ensembles, you need to create a scheme to sample those ensembles. This is done by the `MoveStrategy` and `MoveScheme` objects.

OpenPathSampling uses a simple default scheme for any network, in which first you choose a type of move to do (shooting, replica exchange, etc), and then you choose a specific instance of that move type (i.e., which ensembles to use). This default scheme works for most cases, but you might find yourself in a situation where the default scheme isn't very efficient, or where you think you have an idea for a more efficient scheme. OpenPathSampling makes it easy to modify the underlying move scheme.

#### Definitions of terms

* **move scheme**: for a given simulation, the move scheme is the "move decision tree". Every step of the MC is done by starting with some root move, and tracing a series of decision points to generate (and then accept) a trial.
* **move strategy**: a general approach to building a move scheme (or a subset thereof). SRTIS is a move strategy. Nearest-neighbor replica exchange is a move strategy. All-replica exchange is a move strategy.

So we use "strategy" to talk about the general idea, and "scheme" to talk about a specific implementation of that idea. This document will describe both how to modify the default scheme for one-time modifications and how to develop new move strategies to be re-used on many problems.

## `MoveScheme`s are built from `MoveStrategy` objects

In the end, you must give your `PathSimulator` object a single `MoveScheme`. However, this scheme might involve several different strategies (for example, whether you want to do one-way shooting or two-way shooting is one strategy decision, and it each can be combined with either nearest-neightbor replica exchange strategy or all-replica exchange strategy: these strategy decisions are completely independent.)


### Creating a strategy

A strategy should be thought of as a way to either add new `PathMover`s to a `MoveScheme` or to change those `PathMover`s which already exist in some way.

Every `MoveStrategy` therefore has an `ensembles` parameter. If the `ensembles` parameter is not given, it is assumed that the user intended all normal ensembles in the scheme's transitions. Every strategy also has an initialization parameter called `group`. This defines the "category" of the move. There are several standard categories (described below), but you can also create custom categories (some examples are given later).

Finally, there is another parameter which can be given in the initialization of any strategy, but which *must* be given as a named parameter. This is `replace`, which is a boolean stating whether the movers created using this should replace those in the scheme at this point.


### Strategy groups

Intuitively, we often think of moves in groups: the shooting moves, the replica exchange moves, etc. For organizational and analysis purposes, we include that structure in the `MoveScheme`, and each `MoveStrategy` must declare what groups it applies to. OpenPathSampling allows users to define arbitrary groups (using strings as labels). 


### Strategy levels


### Applying the strategy to a move scheme

To add a strategy to the move scheme, you use `MoveScheme`'s `.append()` function. This function can take two arguments: the list of items to append (which is required) and the `levels` associated with each item.

## Combination strategies

OpenPathSampling provides a few shortcuts to strategies which combine several substrategies into a whole.

### `DefaultMoveStrategy`

The `DefaultMoveStrategy` converts the move scheme to one which follows the default OpenPathSampling behavior.

TODO: note that this isn't always the same as the default scheme you get from an empty move scheme. If other movers exist, they are converted to the default strategy. So if you added movers which are not part of the default for your network, they will still get included in the scheme.

### `SingleReplicaStrategy`

The `SingleReplicaStrategy` converts all replica exchanges to ensemble hops (`bias` parameter required). It then reshapes the move decision tree so that is organized by ensemble, TODO

## Examples of practical uses

In the examples above, we saw how to change from nearest neighbor replica exchange to all (in-set) replica exchange, and we saw how to switch to a single replica move strategy. The next few ideas are similar

### First crossing shooting point selection for some ensembles

For ensembles which are far from the state, sometimes uniform shooting point selection doesn't work. If the number of frames inside the interface is much larger than the number outside the interface, ???

The downside of this approach is that the paths decorrelate much more slowly, since only certain points are allowed for shooting. So while it may be necessary to do it for outer interfaces, doing the same for inner interfaces may slow convergence for them. The trick here is that we apply the first crossing shooting point selection only to the outermost interfaces.

### Two different kinds of shooting for one ensemble

In importance sampling approaches like TIS, you're seeking a balance between two sampling goals. On the one hand, most of space has a negligible (or zero) contribution to the property being measured, so you don't want your steps to be so large that your trials are never accepted. On the other hand, if you make very small steps, it takes a long time to diffuse through the important region (i.e., to decorrelate).

One approach which could be used to fix this would be to allow two different kinds of moves: one which makes small changes with a relatively high acceptance probability to get accepted samples, and one which makes larger changes in an attempt to decorrelate.

This section will show you how to do that by adding a `small_step_shooting` group which does uses the first crossing shooting point selection. (In reality, a better way to get this effect would be to use the standard one-way shooting to do the small steps, and use two-way shooting -- not yet implemented -- to get the larger steps.)

### RepEx-Shoot-RepEx

You might also be interested in trying ???

This particular example also shows you how to implement such features. It includes both implementation of a custom `PathMover` and a custom `MoveStrategy`.